# Spark with MLRun example

This example notebook demonstrates how to execute a spark job with MLRun.

Our spark job is a generic ETL job which pulls data from user-defined data sources, applies a SQL query on top of them, and writes the result to a user defined destination.

The definition of the input-sources should be according to : https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader

The definition of the output destination should be according to :
https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameWriter

In [1]:
import os
from os.path import isfile, join
from mlrun import new_function, NewTask, mlconf

#Set the mlrun database/api
mlconf.dbpath = 'http://mlrun-api:8080'

#Set the pyspark script path
V3IO_WORKING_DIR = os.getcwd().replace('/User','/v3io/'+os.getenv('V3IO_HOME'))
V3IO_SCRIPT_PATH = V3IO_WORKING_DIR+'/spark-function.py'
V3IO_NOTEBOOK_PATH = V3IO_WORKING_DIR+'/spark-function.ipynb'

## Define a task (job parameters)

In [2]:
#Define a dict of input data sources
DATA_SOURCES = {'family' :
                {'format': 'jdbc',
                 'url': 'jdbc:mysql://mysql-rfam-public.ebi.ac.uk:4497/Rfam',
                 'dbtable': 'Rfam.family',
                 'user': 'rfamro',
                 'password': '',
                 'driver': 'com.mysql.jdbc.Driver'},
                'full_region':
                {'format': 'jdbc',
                 'url': 'jdbc:mysql://mysql-rfam-public.ebi.ac.uk:4497/Rfam',
                 'dbtable': 'Rfam.full_region',
                 'user': 'rfamro',
                 'password': '',
                 'driver': 'com.mysql.jdbc.Driver'}
               }

#Define a query to execute on the input data sources
QUERY = 'SELECT family.*, full_region.evalue_score from family INNER JOIN full_region ON family.rfam_acc = full_region.rfam_acc  LIMIT 10'

#Define the output destination
WRITE_OPTIONS = {'format': 'io.iguaz.v3io.spark.sql.kv',
                'mode': 'overwrite',
                'key': 'rfam_id',
                'path': 'v3io://users/admin/frommysql'}

#Create a task execution with parameters
PARAMS = {'data_sources': DATA_SOURCES,
         'query': QUERY,
         'write_options': WRITE_OPTIONS}

SPARK_TASK = NewTask('marcelo-spark',params=PARAMS)

#### Download mysql driver

In [3]:
!wget https://repo1.maven.org/maven2/mysql/mysql-connector-java/8.0.19/mysql-connector-java-8.0.19.jar

--2020-02-19 21:10:45--  https://repo1.maven.org/maven2/mysql/mysql-connector-java/8.0.19/mysql-connector-java-8.0.19.jar
Resolving repo1.maven.org (repo1.maven.org)... 151.101.248.209
Connecting to repo1.maven.org (repo1.maven.org)|151.101.248.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2356711 (2.2M) [application/java-archive]
Saving to: ‘mysql-connector-java-8.0.19.jar’

mysql-connector-jav 100%[===================>]   2.25M  5.88MB/s    in 0.4s    

2020-02-19 21:10:45 (5.88 MB/s) - ‘mysql-connector-java-8.0.19.jar’ saved [2356711/2356711]



## Run locally (in the notebook or attached Spark service)

In [4]:
#Get the list of the dpendency jars
V3IO_JARS_PATH = '/igz/java/libs/'
DEPS_JARS_LIST = [join(V3IO_JARS_PATH, f) for f in os.listdir(V3IO_JARS_PATH) 
                  if isfile(join(V3IO_JARS_PATH, f)) and f.startswith('v3io-') and f.endswith('.jar')]

DEPS_JARS_LIST.append(V3IO_WORKING_DIR + '/mysql-connector-java-8.0.19.jar')

In [5]:
#Create MLRun function which runs locally in a passthrough mode (since we use spark-submit)
#local_spark_fn = new_function(kind='local', mode = 'pass',
#                              command= f"spark-submit --jars {','.join(DEPS_JARS_LIST)} {V3IO_SCRIPT_PATH}")

#Run the function with a task
local_spark_fn = new_function(kind='local', mode = 'pass',
                              command= f"jupyter nbconvert --ExecutePreprocessor.timeout=600 --to notebook {V3IO_NOTEBOOK_PATH}")


local_spark_fn.run(SPARK_TASK)

[mlrun] 2020-02-19 21:10:45,831 starting run marcelo-spark uid=42a64e4db31f4faf839511efa90a23a0  -> http://mlrun-api:8080
2020-02-19 21:10:47 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
[mlrun] 2020-02-19 21:12:26,691 logging run results to: http://mlrun-api:8080
[mlrun] 2020-02-19 21:12:26,752 !@!@!@!@!@ Getting env variables
2020-02-19 21:12:27 INFO  SparkContext:54 - Running Spark version 2.3.1
2020-02-19 21:12:27 INFO  SparkContext:54 - Submitted application: Spark function
2020-02-19 21:12:27 INFO  SecurityManager:54 - Changing view acls to: iguazio
2020-02-19 21:12:27 INFO  SecurityManager:54 - Changing modify acls to: iguazio
2020-02-19 21:12:27 INFO  SecurityManager:54 - Changing view acls groups to: 
2020-02-19 21:12:27 INFO  SecurityManager:54 - Changing modify acls groups to: 
2020-02-19 21:12:27 INFO  SecurityManager:54 - SecurityManager: authentication disabled; ui acls disabled; users  w

uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...0a23a0,0,Feb 19 21:12:26,completed,marcelo-spark,kind=localowner=marcelohost=jupyter-c6luqhyljd-mzpcs-b485d49fb-qjlcf,,"data_sources={'family': {'format': 'jdbc', 'url': 'jdbc:mysql://mysql-rfam-public.ebi.ac.uk:4497/Rfam', 'dbtable': 'Rfam.family', 'user': 'rfamro', 'password': '', 'driver': 'com.mysql.jdbc.Driver'}, 'full_region': {'format': 'jdbc', 'url': 'jdbc:mysql://mysql-rfam-public.ebi.ac.uk:4497/Rfam', 'dbtable': 'Rfam.full_region', 'user': 'rfamro', 'password': '', 'driver': 'com.mysql.jdbc.Driver'}}query=SELECT family.*, full_region.evalue_score from family INNER JOIN full_region ON family.rfam_acc = full_region.rfam_acc LIMIT 10write_options={'format': 'io.iguaz.v3io.spark.sql.kv', 'mode': 'overwrite', 'key': 'rfam_id', 'path': 'v3io://users/admin/frommysql'}",,


to track results use .show() or .logs() or in CLI: 
!mlrun get run 42a64e4db31f4faf839511efa90a23a0 --project default , !mlrun logs 42a64e4db31f4faf839511efa90a23a0 --project default
[mlrun] 2020-02-19 21:12:57,793 run executed, status=completed


## Run as a job on the Kubernetes cluster

In [6]:
#Create MLRun function to run the spark-job on the kubernetes cluster
serverless_spark_fn = new_function(kind='spark', image='urihoenig/spark-app:2.4.4-2.9.0-0.0.3', 
                                   command=f'local://{V3IO_SCRIPT_PATH}', name='my-spark-func')
serverless_spark_fn.with_limits(mem="1g")
serverless_spark_fn.with_requests(cpu=1)
serverless_spark_fn.with_igz_spark(igz_version='2.8_b3506_20191217042239')
#serverless_spark_fn.with_igz_spark(igz_version='2.5_b2782_20190922162238')
serverless_spark_fn.spec.deps['jars'] += [V3IO_WORKING_DIR+'/mysql-connector-java-8.0.19.jar']
#Set number of executors
serverless_spark_fn.spec.replicas = 2
serverless_spark_fn.run(SPARK_TASK,watch=True)

[mlrun] 2020-02-19 21:23:11,132 starting run marcelo-spark uid=cf553af35dec497ca3ecf0c30adfa959  -> http://mlrun-api:8080
++ id -u
+ myuid=1000
++ id -g
+ mygid=1000
+ set +e
++ getent passwd 1000
+ uidentry=iguazio:x:1000:1000::/igz:/bin/bash
+ set -e
+ '[' -z iguazio:x:1000:1000::/igz:/bin/bash ']'
+ SPARK_K8S_CMD=driver-py
+ case "$SPARK_K8S_CMD" in
+ shift 1
+ SPARK_CLASSPATH=':/spark/jars/*'
+ env
+ grep SPARK_JAVA_OPT_
+ sort -t_ -k4 -n
+ sed 's/[^=]*=\(.*\)/\1/g'
+ readarray -t SPARK_EXECUTOR_JAVA_OPTS
+ '[' -n '' ']'
+ '[' -n file:///igz/java/libs/v3io-py.zip ']'
+ PYTHONPATH=:file:///igz/java/libs/v3io-py.zip
+ PYSPARK_ARGS=
+ '[' -n '' ']'
+ R_ARGS=
+ '[' -n '' ']'
+ '[' 2 == 2 ']'
++ python -V
+ pyv='Python 3.6.8'
+ export PYTHON_VERSION=3.6.8
+ PYTHON_VERSION=3.6.8
+ export PYSPARK_PYTHON=python
+ PYSPARK_PYTHON=python
+ export PYSPARK_DRIVER_PYTHON=python
+ PYSPARK_DRIVER_PYTHON=python
+ case "$SPARK_K8S_CMD" in
+ CMD=("$SPARK_HOME/bin/spark-submit" --conf "spark.driver.bi

uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...dfa959,0,Feb 19 21:23:11,completed,marcelo-spark,kind=sparkowner=admin,,"data_sources={'family': {'dbtable': 'Rfam.family', 'driver': 'com.mysql.jdbc.Driver', 'format': 'jdbc', 'password': '', 'url': 'jdbc:mysql://mysql-rfam-public.ebi.ac.uk:4497/Rfam', 'user': 'rfamro'}, 'full_region': {'dbtable': 'Rfam.full_region', 'driver': 'com.mysql.jdbc.Driver', 'format': 'jdbc', 'password': '', 'url': 'jdbc:mysql://mysql-rfam-public.ebi.ac.uk:4497/Rfam', 'user': 'rfamro'}}query=SELECT family.*, full_region.evalue_score from family INNER JOIN full_region ON family.rfam_acc = full_region.rfam_acc LIMIT 10write_options={'format': 'io.iguaz.v3io.spark.sql.kv', 'key': 'rfam_id', 'mode': 'overwrite', 'path': 'v3io://users/admin/frommysql'}",,


to track results use .show() or .logs() or in CLI: 
!mlrun get run cf553af35dec497ca3ecf0c30adfa959 --project default , !mlrun logs cf553af35dec497ca3ecf0c30adfa959 --project default
[mlrun] 2020-02-19 21:25:22,444 run executed, status=completed
